In [12]:
import  pandas as pd
%matplotlib  inline 
import pandas as pd
import numpy as np
from igraph  import Graph as IGraph
def getGraph(GV): 
    '''图的构件，输入的是numpy的数组，如果3表示图是带权重的，否则没有权重
    '''
    if GV.shape[1]==3:
        g=IGraph.TupleList(GV,directed=False,vertex_name_attr='name',weights=True)  ## 构建图
    else:
        g=IGraph.TupleList(GV,directed=True,vertex_name_attr='name')  ## 构建图
    return  g

def getStats(g):
    '''diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_weights'''
    return  [g.diameter(),g.density(),g.clique_number(),g.average_path_length(),\
            g.vcount(),\
            g.transitivity_undirected(),## 聚集系数
            g.assortativity_degree(directed=False),   ##同配系数
            g.omega(),
            np.array(g.es['weight']).mean()
            ]  
        

        
def GetCommStats(rel_net):
    g=getGraph(rel_net)
    names=g.vs['name']
    weights=g.es['weight']
    cluster=IGraph.community_label_propagation(g,weights='weight').as_cover()
    if data.shape[1]==3:
        pg_df=pd.DataFrame(g.pagerank(weights='weight'),index=names,columns=['pg'])
    else:
        pg_df=pd.DataFrame(g.pagerank(weights='weight'),index=names,columns=['pg'])

    cluster=[line for line in  list(cluster)  if line>6]

    columns=['diameter','density','clique_number','average_path_length','nums','coef','pg_mean']
    comm_stats=[]
    for line in cluster:
        pg_mean=pg_df.iloc[line].mean().values[0]
        gs=getStats(g.subgraph(line))+[pg_mean]
        comm_stats.append(gs)
    x=pd.DataFrame(comm_stats,columns=columns)
    return x

def getClusterStats(g,cluster):
    cluster=[line for line in  list(cluster)  if line>6]

    columns=['diameter','density','clique_number','average_path_length','nums','coef']
    comm_stats=[]
    for line in cluster:
        pg_mean=pg_df.iloc[line].mean().values[0]
        gs=getStats(g.subgraph(line))+[pg_mean]
        comm_stats.append(gs)
    x=pd.DataFrame(comm_stats,columns=columns)
    return x

def getCommStats(comm):
    '''获取每个社区的特性'''
    from multiprocessing  import Pool
    pool=Pool(8)
    sub_stats=pool.map(getStats,comm.subgraphs())
    pool.close()
    pool.join()
    columns='diameter,density,cliqu_number,avg_path_length,vcount,coef,assortativity,omega,mean_rel'.split(',')
#     sub_stats=map(getStats,rel_comm_lpa.subgraphs())
    sub_stats=pd.DataFrame(sub_stats,columns=columns)
    return sub_stats
def GetGVlues(fname):
    df=pd.read_csv(fname,index_col=0)
    df=df[df.cust_a!=df.cust_b]
    print df.shape[0]
    g=getGraph(df.values)
    return g

In [17]:

cust_user_info=pd.read_csv("comm_detect/chapter1data/user_info0330.csv",index_col=0)



In [3]:
def get_PersonInfo(contact_num):
    contact_num=cust_user_info.join(contact_num,how='inner',rsuffix='x')
    contact_num["age"]=2016-contact_num.id_no.str.slice(6,10).astype("float")
    ##奇数为男性，偶数为女性
    contact_num["sex"]=contact_num.id_no.str.slice(16,17).bfill().replace('','0').astype("float")%2
    contact_num.sex.replace({1.0:'male',0.0:'female'},inplace=True)
    contact_numa=contact_num[(contact_num.age>10) &(contact_num.age<60)]
    return  contact_num

In [19]:
cust_user_info.dropna(inplace=True)

In [23]:
cust_user_info.drop_duplicates('id',inplace=True)`

In [41]:
cust_user_info=cust_user_info[cust_user_info.id_no.str.len()==18]

In [43]:
cust_user_info.head(3)

,id,id_no
0,173464,330324198707164O1.
1,7798526,21010519880311(!0A
3,9273658,340303199404174.1!


In [44]:
cust_user_info['age']=2016-cust_user_info.id_no.str.slice(6,10).astype("float")

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [45]:
cust_user_info['sex']=cust_user_info.id_no.str.slice(16,17).bfill().replace('','0').astype("float")%2

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [46]:
cust_user_info.sex.replace({1.0:'male',0.0:'female'},inplace=True)


/opt/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [50]:
cust_user_info.drop(['id_no'],axis=1,inplace=True)

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [52]:
cust_user_info.to_csv('comm_detect/chapter1data/user_info.csv')

In [30]:
dd=cust_user_info.id_no.str.slice(6,10)

In [48]:
cust_user_info.set_index('id',inplace=True)

In [51]:
cust_user_info.head(3)

,age,sex
id,,
173464,29.0,male
7798526,28.0,female
9273658,22.0,male
